In [ ]:
# ЧЕРНОВОЙ ВАРИАНТ ПО САММАРИЗАЦИИ ВИДЕО БЕЗ ТИТРОВ В COLAB c ПРОВЕРКОЙ НА ДУБЛИ и ОПЕЧАТКИ

# Можете копию сделать, поиграться, только на Google Disk надо в KIA_VIDEO папке CURRENT.docx файл создать, дать права на просмотр и ссылку подставить вместо моей в переменную CURRENT=.

# # Для использования на гугл диске в меню доступные мне найдите V1 и замонтируйте как папку к себе на Мой Диск


from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip -q install langchain openai chromadb

In [ ]:
import csv

from langchain.text_splitter import MarkdownHeaderTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter

headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
]

name = ""
db_splits = []
markdown_document = ""
with codecs.open("/content/drive/My Drive/V1/KIA (Транскрибация)/parser.txt", "r", encoding=encoding) as fs:
    while True:
        fs_line = fs.readline().encode('utf-8', 'ignore').decode(encoding)
        if not fs_line: break
        name = re.match('<[^>]*>', fs_line)
        name = name.encode(encoding, 'ignore').decode('utf-8') if name else ""
        markdown_document += re.sub('<[^>]*>', '', fs_line).encode(encoding, 'ignore').decode('utf-8')
# MD splits
markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
md_header_splits = markdown_splitter.split_text(markdown_document)

# Char-level splits
chunk_size = 1024
chunk_overlap = 30
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size, chunk_overlap=chunk_overlap
)

# Split
splits = text_splitter.split_documents(md_header_splits)
db_splits = [split.page_content for split in splits]

In [ ]:
!mkdir -p "/content/drive/My Drive/KIA_VIDEO/"

In [ ]:
WHISPER_TEXTS={"11.2.3" : ["https://drive.google.com/file/d/", "1KY134ZBEkitRkrpdfH_ISaef_uoUaMMu", "/view?usp=sharing"]}

In [ ]:
MY_FIXES="""киос кэ	киоске
тридын	трейд ин
кеоптима	киа оптима
марк укея	марку киа
кеа оптима	киа оптима
кеа	киа
кестингер	киа стингер"""
MY_PROMPT="""
У тебя есть подробная информация по работе с ним.
Тебе задает вопрос пользователь, дай ему информацию, опираясь на предоставленные материалы.
Отвечай максимально точно и используй только информацию из документов, не добавляй ничего своего.
Описывай текст словами клиента. Ты должен дать ответ в форме рекомендации. Cтарайся использовать безличные предложения. Удали из своего ответа редакторов, авторов. Не называй в ответе фамилий и имен.
Предварительно исправь в документе все опечатки: """ + ",".join([" замени \"" + line.split("\t")[0] + "\" на \"" + line.split("\t")[1] + "\"" for line in MY_FIXES.split("\n")]) + """.
Документ с информацией для ответа пользователю:
"""

In [ ]:
#@title Установка библиотек
!pip  install  tiktoken  langchain openai chromadb gspread oauth2client nltk pydantic==1.10.8

In [ ]:
!pip install python-docx

In [ ]:
#@title Импорт библиотек
import gdown
from langchain.llms import OpenAI
from langchain.docstore.document import Document
import requests
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
import pathlib
import subprocess
import tempfile
import ipywidgets as widgets
import os
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import re
import getpass
import os
import openai
import tiktoken

In [ ]:
# Получение ключа API от пользователя и установка его как переменной окружения
openai_key = getpass.getpass("OpenAI API Key:")
os.environ["OPENAI_API_KEY"] = openai_key
openai.api_key = openai_key

In [ ]:
# Переводим текстовый файл в word
import requests
import docx

def convert_txt_to_docx(file_ids, output_path):
    response = requests.get("https://docs.google.com/uc?export=download&id="+file_ids[1])
    encoding = response.encoding
    content = response.text

    doc = docx.Document()
    doc.add_paragraph(content.encode(encoding, 'ignore').decode('utf-8'))

    doc.save(output_path)

    print(encoding)

In [ ]:
# Монтируем каталог на гугл диске и в нем файлу CURRENT.docx даем права на чтение всем, у кого ссылка
convert_txt_to_docx(WHISPER_TEXTS["11.2.3"], "/content/drive/My Drive/KIA_VIDEO/CURRENT.docx")
CURRENT="https://docs.google.com/document/d/1EMnFE4K4cKzbFAh9fKYSUiHJRm4ib9ao/edit"

In [ ]:
# Нарезаем CURRENT.docx на чанки
def split_text(text, max_count, count_type, verbose=0):
    # Функция для подсчета количества слов в фрагменте
    def num_words(fragment):
        return len(fragment.split())

    # Функция для подсчета количества токенов в фрагменте
    def num_tokens(fragment):
        return num_tokens_from_string(fragment, "cl100k_base")

    # Разделение текста на фрагменты, исключая теги HTML
    fragments = [fragment.strip() for fragment in re.split(r"<[^>]+>|[\ufeff]", text) if fragment.strip()]

    # Выбор функции подсчета длины в зависимости от типа подсчета
    length_function = num_words if count_type == "words" else num_tokens

    # Создание объекта разделителя текста
    splitter = RecursiveCharacterTextSplitter(chunk_size=max_count, chunk_overlap=0, length_function=length_function)

    # НОВОЕ - фильтруем очень похожие чанки, если они был в базе parser.txt
    # YOUR CODE HERE

    # Список для хранения фрагментов текста
    source_chunks = []

    # Обработка каждого фрагмента текста
    for fragment in fragments:
        if verbose:
            # Вывод количества слов/токенов в фрагменте, если включен режим verbose
            count = length_function(fragment)
            print(f"{count_type} in text fragment = {count}\n{'-' * 5}\n{fragment}\n{'=' * 20}")

        # Разбиение фрагмента текста на части заданной длины с помощью разделителя
        # и добавление каждой части в список source_chunks
        source_chunks.extend(Document(page_content=chunk, metadata={}) for chunk in splitter.split_text(fragment))

    # Возвращение списка фрагментов текста
    return source_chunks


def create_embedding(data, max_count, count_type):
    def num_tokens_from_string(string: str, encoding_name: str) -> int:
      """Возвращает количество токенов в строке"""
      encoding = tiktoken.get_encoding(encoding_name)
      num_tokens = len(encoding.encode(string))
      return num_tokens

    source_chunks = []

    source_chunks = split_text(text=data, max_count=max_count, count_type=count_type, verbose=0)

    # Создание индексов документа
    search_index = Chroma.from_documents(source_chunks, OpenAIEmbeddings(), )

    count_token = num_tokens_from_string(' '.join([x.page_content for x in source_chunks]), "cl100k_base")
    print('\n ===========================================: ')
    print('Количество токенов в документе :', count_token)
    print('ЦЕНА запроса:', 0.0004*(count_token/1000), ' $')
    return search_index

def load_search_indexes(url: str, max_count, count_type) -> str:
    # Extract the document ID from the URL
    match_ = re.search('/document/d/([a-zA-Z0-9-_]+)', url)
    if match_ is None:
        raise ValueError('Invalid Google Docs URL')
    doc_id = match_.group(1)

    # Download the document as plain text
    response = requests.get(f'https://docs.google.com/document/d/{doc_id}/export?format=txt')
    response.raise_for_status()
    text = response.text
    return create_embedding(text, max_count=max_count, count_type=count_type)


In [ ]:
# Готовим для CURRENT.docx эмбединги
def num_tokens_from_messages(messages, model="gpt-3.5-turbo-0301"):
    """Returns the number of tokens used by a list of messages."""
    try:
        encoding = tiktoken.encoding_for_model(model)
    except KeyError:
        encoding = tiktoken.get_encoding("cl100k_base")
    if model == "gpt-3.5-turbo-0301":  # note: future models may deviate from this
        num_tokens = 0
        for message in messages:
            num_tokens += 4  # every message follows <im_start>{role/name}\n{content}<im_end>\n
            for key, value in message.items():
                num_tokens += len(encoding.encode(value))
                if key == "name":  # if there's a name, the role is omitted
                    num_tokens += -1  # role is always required and always 1 token
        num_tokens += 2  # every reply is primed with <im_start>assistant
        return num_tokens
    else:
        raise NotImplementedError(f"""num_tokens_from_messages() is not presently implemented for model {model}.""")

def insert_newlines(text: str, max_len: int = 170) -> str:
    words = text.split()
    lines = []
    current_line = ""
    for word in words:
        if len(current_line + " " + word) > max_len:
            lines.append(current_line)
            current_line = ""
        current_line += " " + word
    lines.append(current_line)
    return "\n".join(lines)

def answer_index(system, topic, search_index, temp = 1, verbose = 0, top_similar_documents = 5):

    #Выборка документов по схожести с вопросом
    docs = search_index.similarity_search(topic, k=top_similar_documents)
    if (verbose): print('\n ===========================================: ')
    message_content = re.sub(r'\n{2}', ' ', '\n '.join([f'\nОтрывок документа №{i+1}\n=====================' + doc.page_content + '\n' for i, doc in enumerate(docs)]))
    if (verbose): print('message_content :\n ======================================== \n', message_content)

    messages = [
      {"role": "system", "content": system + f"{message_content}"},
      {"role": "user", "content": topic}
      ]

    # example token count from the function defined above
    if (verbose): print('\n ===========================================: ')
    if (verbose): print(f"{num_tokens_from_messages(messages, 'gpt-3.5-turbo-0301')} токенов использовано на вопрос")

    completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=messages,
    temperature=temp
    )
    if (verbose): print('\n ===========================================: ')
    if (verbose): print(f'{completion["usage"]["total_tokens"]} токенов использовано всего (вопрос-ответ).')
    if (verbose): print('\n ===========================================: ')
    if (verbose): print('ЦЕНА запроса с ответом :', 0.002*(completion["usage"]["total_tokens"]/1000), ' $')
    if (verbose): print('\n ===========================================: ')

    res = insert_newlines(completion.choices[0].message.content)
    print('ОТВЕТ : \n', )

    return res
    # return completion

In [ ]:
# Просим ChatGPT ответить на вопрос "Кратко перескажи текст:"
max_count = 110
count_type = "words"
top_similar_documents = 5
# Документ "инструкция по эксплуатации"
manual_index = load_search_indexes(CURRENT, max_count=max_count, count_type=count_type)
ans=answer_index(
    MY_PROMPT,
    'Кратко перескажи текст.',                                              # напишите вопрос для проверки инструкции
    manual_index,
    verbose = 1,
    top_similar_documents = top_similar_documents
)

In [ ]:
# Сохраняем ответ на гугл диск
def save_ans_to_docx(ans, output_path):
    content = ans

    doc = docx.Document()
    doc.add_paragraph(content)
    doc.save(output_path)
save_ans_to_docx(ans, "/content/drive/My Drive/V1/KIA (Транскрибация)/" + "_11.2.3" + ".docx")